In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


In [2]:
def thetaPhi():
    '''returns random theta and phi values'''
    theta = np.random.uniform(0,2*np.pi)
    phi = np.random.uniform(0,np.pi)
    return theta, phi

def momenta(mag, theta, phi):
    '''given an input magnitude, theta and phi the cartesian momenta are returned'''
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz
    

def wArr(num):
    '''just for making numpy array of stationary w given the number of events'''
    momenta = np.zeros([num,4])
    momenta[:,3] = wMass
    
    return momenta

def vecToArr(vec):
    '''converts lorentzvector to array of components'''
    arr = np.zeros([4])
    arr[0] = vec.Px()
    arr[1] = vec.Py()
    arr[2] = vec.Pz()
    arr[3] = vec.E()
    
    return arr

In [3]:
def backToBack(beta):
    '''from a given beta (ratio v/c) of the initial particles produces numpy array of 4 momenta of
    w pair with one hadronic decay and one leptonic decay'''
    event = np.zeros([5,4])
    # making back to back w bosons (stationary)
    wMass = 80.3
    
    wmag = 3 # ??
    
    theta, phi = thetaPhi()
    Px, Py, Pz = momenta(wmag, theta, phi)

    w1 = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, wMass)
    w2 = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, wMass)

    #boosting the Ws
    beta = beta
    boostedW1 = r.Math.VectorUtil.boostZ(w1, beta)
    w1Arr = vecToArr(boostedW1)
    boostedW2 = r.Math.VectorUtil.boostZ(w2, beta)
    w2Arr = vecToArr(boostedW2)

    # have to make them into tlorentzvectors for the boostvector function to work
    boow1 = r.TLorentzVector()
    boow1.SetPxPyPzE(boostedW1.Px(), boostedW1.Py(), boostedW1.Pz(), boostedW1.E())
    boow2 = r.TLorentzVector()
    boow2.SetPxPyPzE(boostedW2.Px(), boostedW2.Py(), boostedW2.Pz(), boostedW2.E())

    #making boost vectors for the daughter particles of each w
    boost1 = boow1.BoostVector()
    boost2 = boow2.BoostVector()


    #making the hadronic decay particles this will use w1 as its parent
    #in w1 rest frame to start
    ej = wMass/2
    thetaj, phij = thetaPhi()
    Px, Py, Pz = momenta(ej, thetaj, phij)
    j1 = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, ej)
    j2 = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, ej)

    # boosting jets to frame of W1
    j1Boost = r.Math.VectorUtil.boost(j1, boost1)
    j1Arr = vecToArr(j1Boost)
    j2Boost = r.Math.VectorUtil.boost(j2, boost1)
    j2Arr = vecToArr(j2Boost)

    # making leptonic decay particles using W2 as parent
    # w2 rest frame to start
    eE = wMass / 2
    thetae, phie = thetaPhi()
    Px, Py, Pz = momenta(eE, thetae, phie)
    e = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, eE)
    #n = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, eE)

    #boosting e and n to frame of W2
    eBoost = r.Math.VectorUtil.boost(e, boost2)
    eArr = vecToArr(eBoost)
    #nBoost = r.Math.VectorUtil.boost(n, boost2)
    #nArr = vecToArr(nBoost)

    #putting all particles into one array
    event[0] = w1Arr
    event[1] = w2Arr
    event[2] = j1Arr
    event[3] = j2Arr
    event[4] = eArr
    #event[5] = nArr
    
    return event

In [4]:
print(backToBack(0.3))

[[  1.57926429  -2.3751912   26.22779973  84.46965779]
 [ -1.57926429   2.3751912   24.2785637   83.88488698]
 [ 11.75350131 -38.59493316  23.77227555  46.82771341]
 [-10.17313372  36.21808262   2.47384725  37.70095594]
 [  3.55575056  12.10376141 -27.89048182  30.6108376 ]]


In [5]:
def  generateEvents(numEv, beta):
    ''' Generates number of different events.
    '''
    
    events = np.zeros([numEv,5,4])
    
    for i in range(numEv):
        events[i] = backToBack(beta)
     
    return events    

In [6]:
numberOfEvents = 10
beta = 0.1
events = generateEvents(numberOfEvents,beta)

In [7]:
events[0]

array([[ 6.45706731e-02,  5.34239115e-02,  5.05651692e+00,
         8.04031429e+01],
       [-6.45706731e-02, -5.34239115e-02,  1.10843904e+01,
         8.10059302e+01],
       [ 1.06089983e+01, -3.07045473e+01,  2.61502703e+01,
         4.17031978e+01],
       [-1.05443825e+01,  3.07580085e+01, -2.10902209e+01,
         3.87561157e+01],
       [-2.76930559e+01, -6.59252124e+00,  3.41702587e+01,
         4.44744113e+01]])

In [8]:
tree = r.TTree("tree", "initial random e and n components")

w1Branch = np.zeros(4)
w2Branch = np.zeros(4)
jet1Branch = np.zeros(4)
jet2Branch = np.zeros(4)
eBranch = np.zeros(4)

tree.Branch("w1Boson",w1Branch,"w1Array[4]/D")
tree.Branch("w2Boson",w2Branch,"w2Array[4]/D")
tree.Branch("jet1",jet1Branch,"jet1Array[4]/D")
tree.Branch("jet2",jet2Branch,"jet2Array[4]/D")
tree.Branch("electron",eBranch,"elArray[4]/D")

In [9]:
# Filling values into TTree
for i in range(numberOfEvents):
    for j in range(4):
        w1Branch[j] = events[i,0,j]
        w2Branch[j] = events[i,1,j]
        jet1Branch[j] = events[i,2,j]
        jet2Branch[j] = events[i,3,j]
        eBranch[j] = events[i,4,j]
    
    tree.Fill()

In [13]:
file = r.TFile("events2W.root","recreate")
tree.Write()
file.Close

In [14]:
tree.Scan()

40

***********************************************************************************
*    Row   * Instance * w1Boson.w * w2Boson.w * jet1.jet1 * jet2.jet2 * electron. *
***********************************************************************************
*        0 *        0 * 0.0645706 * -0.064570 * 10.608998 * -10.54438 * -27.69305 *
*        0 *        1 * 0.0534239 * -0.053423 * -30.70454 * 30.758008 * -6.592521 *
*        0 *        2 * 5.0565169 * 11.084390 * 26.150270 * -21.09022 * 34.170258 *
*        0 *        3 * 80.403142 * 81.005930 * 41.703197 * 38.756115 * 44.474411 *
*        1 *        0 * -0.567484 * 0.5674849 * -4.686208 * 4.1183267 * 7.9521592 *
*        1 *        1 * -0.600520 * 0.6005204 * -0.809739 * 0.2087991 * 15.679958 *
*        1 *        2 * 5.1719454 * 10.968961 * 42.577513 * -37.40195 * -31.11260 *
*        1 *        3 * 80.414685 * 80.994387 * 42.842279 * 37.628585 * 35.736427 *
*        2 *        0 * -0.821853 * 0.8218536 * 15.991534 * -16.81396 * 39.9

Type <CR> to continue or q to quit ==> 